In [4]:
import pandas as pd

In [5]:
# Load the datasets
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

In [6]:
# Inspect the first few rows of each dataset
print(customers_df.head())
print(products_df.head())
print(transactions_df.head())

  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15
  ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    ActiveWear Smartwatch  Electronics  346.30
2      P003  ComfortLiving Biography        Books   44.12
3      P004            BookWorld Rug   Home Decor   95.69
4      P005          TechPro T-Shirt     Clothing  429.31
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3       

In [7]:
# Convert SignupDate to datetime
customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'])

In [8]:
# Merge customer and transaction data on CustomerID
transactions_with_customer = pd.merge(transactions_df, customers_df, on='CustomerID', how='left')

In [9]:
# Merge with product data on ProductID
transactions_with_full_info = pd.merge(transactions_with_customer, products_df, on='ProductID', how='left')

In [10]:
# Inspect the combined dataset
print(transactions_with_full_info.head())

  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x     CustomerName         Region SignupDate  \
0      300.68   300.68   Andrea Jenkins         Europe 2022-12-03   
1      300.68   300.68  Brittany Harvey           Asia 2024-09-04   
2      300.68   300.68  Kathryn Stevens         Europe 2024-04-04   
3      601.36   300.68  Travis Campbell  South America 2024-04-11   
4      902.04   300.68    Timothy Perez         Europe 2022-03-15   

                       ProductName     Category  Price_y  
0  ComfortLiving Bluetooth Speaker  Electronics   300.68  
1  ComfortLiving Bluetooth Speaker  Elec

In [11]:
# Group by CustomerID to compute customer-level features
customer_features = transactions_with_full_info.groupby('CustomerID').agg(
    total_spent=pd.NamedAgg(column='TotalValue', aggfunc='sum'),
    num_transactions=pd.NamedAgg(column='TransactionID', aggfunc='nunique'),
    num_products=pd.NamedAgg(column='ProductID', aggfunc='nunique'),
    avg_transaction_value=pd.NamedAgg(column='TotalValue', aggfunc='mean')
).reset_index()

In [12]:
# Merge the features with customer data
customer_features = pd.merge(customer_features, customers_df[['CustomerID', 'Region']], on='CustomerID', how='left')

In [13]:
# Inspect the features
print(customer_features.head())

  CustomerID  total_spent  num_transactions  num_products  \
0      C0001      3354.52                 5             5   
1      C0002      1862.74                 4             4   
2      C0003      2725.38                 4             4   
3      C0004      5354.88                 8             8   
4      C0005      2034.24                 3             3   

   avg_transaction_value         Region  
0                670.904  South America  
1                465.685           Asia  
2                681.345  South America  
3                669.360  South America  
4                678.080           Asia  


In [14]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [15]:
# Select relevant features for similarity calculation (excluding CustomerID)
features = customer_features[['total_spent', 'num_transactions', 'num_products', 'avg_transaction_value']]

In [16]:
# Normalize the features if needed (you can use StandardScaler from sklearn)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [17]:
# Compute the cosine similarity matrix
similarity_matrix = cosine_similarity(features_scaled)

In [18]:
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])
print(similarity_df.head())

CustomerID     C0001     C0002     C0003     C0004     C0005     C0006  \
CustomerID                                                               
C0001       1.000000  0.681909  0.137497 -0.000631  0.104379 -0.817502   
C0002       0.681909  1.000000  0.726043 -0.649737  0.713370 -0.574811   
C0003       0.137497  0.726043  1.000000 -0.990094  0.999224  0.145356   
C0004      -0.000631 -0.649737 -0.990094  1.000000 -0.994545 -0.243356   
C0005       0.104379  0.713370  0.999224 -0.994545  1.000000  0.162666   

CustomerID     C0007     C0008     C0009     C0010  ...     C0191     C0192  \
CustomerID                                          ...                       
C0001      -0.333941  0.386237  0.624453  0.709473  ...  0.919641  0.624733   
C0002       0.251699 -0.287615  0.995853  0.998487  ...  0.857196  0.994684   
C0003       0.847614 -0.845892  0.755674  0.687131  ...  0.287739  0.792995   
C0004      -0.892865  0.904547 -0.688581 -0.607304  ... -0.174444 -0.723908   
C0005  

In [27]:
import numpy as np
import pandas as pd

# Define the function to get top N lookalikes
def get_top_lookalikes(customer_idx, similarity_matrix, top_n=3):
    # Get similarity scores for the given customer index
    similarity_scores = similarity_matrix[customer_idx]
    
    # Sort customers by similarity score in descending order (exclude the customer itself)
    sorted_indices = np.argsort(similarity_scores)[::-1][1:top_n+1]  # Exclude the customer itself
    
    # Extract similarity scores for the top N similar customers
    top_scores = similarity_scores[sorted_indices]
    
    return [(idx, score) for idx, score in zip(sorted_indices, top_scores)]

# Create a dictionary for lookalike mapping
lookalike_dict = {}

# Ensure CustomerID is mapped to its corresponding index
customer_id_to_index = {cid: idx for idx, cid in enumerate(customer_features['CustomerID'])}
index_to_customer_id = {idx: cid for cid, idx in customer_id_to_index.items()}

# Get the top 3 lookalikes for customers from C0001 to C0020
for customer_id in customer_features['CustomerID'][:20]:
    customer_idx = customer_id_to_index[customer_id]
    lookalikes = get_top_lookalikes(customer_idx, similarity_matrix, top_n=3)
    
    # Convert indices back to CustomerID
    lookalike_dict[customer_id] = [(index_to_customer_id[idx], score) for idx, score in lookalikes]

# Convert the lookalike dictionary into a DataFrame
lookalike_rows = []
for cust_id, lookalikes in lookalike_dict.items():
    row = {"CustomerID": cust_id}
    for i, (lookalike_id, score) in enumerate(lookalikes):
        row[f"Lookalike_{i+1}_ID"] = lookalike_id
        row[f"Lookalike_{i+1}_Score"] = score
    lookalike_rows.append(row)

lookalike_df = pd.DataFrame(lookalike_rows)

# Save to CSV
lookalike_df.to_csv('Lookalike.csv', index=False)
print(lookalike_df.head())


  CustomerID Lookalike_1_ID  Lookalike_1_Score Lookalike_2_ID  \
0      C0001          C0137           0.996211          C0152   
1      C0002          C0029           0.999535          C0199   
2      C0003          C0178           0.999689          C0005   
3      C0004          C0021           0.999785          C0075   
4      C0005          C0073           0.999667          C0063   

   Lookalike_2_Score Lookalike_3_ID  Lookalike_3_Score  
0           0.981064          C0056           0.948258  
1           0.998585          C0010           0.998487  
2           0.999224          C0073           0.998479  
3           0.999585          C0067           0.999207  
4           0.999401          C0159           0.999295  
